In [8]:
import numpy as np
import itertools
import random
from statistics import stdev
import math
import ast

In [9]:
p_running = 0.9
p_failure = 1 - p_running
path = '/home/dkabe/Model_brainstorming/SAA_Analysis/'

In [10]:
provinces = [3, 6]
plants = 6
dc = 4
a5 = list(itertools.product([1, 0], repeat = provinces[0]))
a6 = list(itertools.product([1, 0], repeat = provinces[1]))
b = list(itertools.product([1, 0], repeat = dc))
mp_loc_encoded = [[0, 0, 1, 1, 2, 2],
                  [0, 1, 2, 3, 4, 5]]

In [4]:
province_disruptions = [[x,y] for x in a6 for y in b]
num_Scenarios = len(province_disruptions)
plant_failure = [[None]*plants for x in range(2**provinces[1])]
for k in range(len(plant_failure)):
    for i in range(plants):
        ind = mp_loc_encoded[1][i]
        plant_failure[k][i] = a6[k][ind]
Scen = [[x,y] for x in plant_failure for y in b]
p_scen = []
for s in range(len(province_disruptions)):
    p_i = (p_running**(np.sum(province_disruptions[s][0]) + np.sum(province_disruptions[s][1])))*(p_failure**(provinces[1] + dc - (np.sum(province_disruptions[s][0]) + np.sum(province_disruptions[s][1]))))
    p_scen.append(p_i)
    
#with open(path + '/Evaluation_Set.txt', 'w') as f:
 #   for item in Scen:
  #      f.write("%s\n" % item)
#f.close()
#np.savetxt("p_scen.txt", p_scen)

In [11]:
N = [50, 100, 150, 200, 250, 300, 350]
batches = 30
Products = 3
Market = 29
dem = np.loadtxt("/home/dkabe/Model_brainstorming/SAA_Analysis/eval_set_demand.txt").reshape(1024,3,29)

In [44]:
random.seed(1)
for num_scen in N:
    for batch in range(batches):
        indices = random.sample(range(len(Scen)), num_scen)
        demand = np.array([dem[index] for index in indices])
        sample_scenarios = [Scen[index] for index in indices]
        p_sample_scenarios = [p_scen[index] for index in indices]
        p_factor = 1/np.sum(p_sample_scenarios)
        p_sample_scenarios = list(map(lambda x: x*p_factor, p_sample_scenarios))  
        with open(path + 'Scenarios/' + str(num_scen) + '_' + str(batch) + '.txt', 'w') as f:
            for item in sample_scenarios:
                f.write("%s\n" % item)
        f.close()
        np.savetxt(path + 'Scen_probabilities/' + 'p_scen_' + str(num_scen) + '_' + str(batch) + '.txt', p_sample_scenarios)
        np.savetxt(path + 'Scen_demand/' + 'demand_' + str(num_scen) + '_' + str(batch) + '.txt', demand.reshape((num_scen*Products, Market)))



In [60]:
counter = 0 
for num_scen in N:
    for batch in range(batches):
        probs = np.loadtxt(path + 'Scen_probabilities/' + 'p_scen_' + str(num_scen) + '_' + str(batch) + '.txt')
        counter += np.sum(probs)

In [61]:
counter

180.0

In [5]:
p1_mean = 137000
p2_mean = 135000
p3_mean = 133000

p1_sd = 2650
p2_sd = 8125
p3_sd = 4000

distribution_mat = [[p1_mean, p1_sd],
                  [p2_mean, p2_sd],
                  [p3_mean, p3_sd]]

In [9]:
# forming evaluation set demand 
demand = np.zeros((1024, Products, Market))
for s in range(len(Scen)):
    demand[s][0] = np.round(np.random.normal(distribution_mat[0][0], distribution_mat[0][1], size = 29))
    demand[s][1] = np.round(np.random.normal(distribution_mat[1][0], distribution_mat[1][1], size = 29))
    demand[s][2] = np.round(np.random.normal(distribution_mat[2][0], distribution_mat[2][1], size = 29))

#np.savetxt("eval_set_demand.txt", demand.reshape((1024*3,29)))

In [34]:
# eval set v2
random.seed(2)
for num_scen in N:
    indices = random.sample(range(len(Scen)), 200)
    scenarios = 25*[Scen[index] for index in indices]
    p_sample_scenarios = 25*[p_scen[index] for index in indices]
    p_factor = 1/np.sum(p_sample_scenarios)
    p_sample_scenarios = list(map(lambda x: x*p_factor, p_sample_scenarios))
    with open(path + 'Eval_Sets/' + str(num_scen) + '_scenarios' + '/evaluation_scenarios.txt', 'w') as f:
      for item in scenarios:
        f.write("%s\n" % item)
    f.close()

    np.savetxt(path + '/Eval_Sets/' + str(num_scen) + '_scenarios' + "/p_scen.txt", p_sample_scenarios)
    


In [36]:
random.seed(2)
for num_scen in N:
    demand = np.zeros((5000, Products, Market))
    for s in range((5000)):
        demand[s][0] = np.round(np.random.normal(distribution_mat[0][0], distribution_mat[0][1], size = 29))
        demand[s][1] = np.round(np.random.normal(distribution_mat[1][0], distribution_mat[1][1], size = 29))
        demand[s][2] = np.round(np.random.normal(distribution_mat[2][0], distribution_mat[2][1], size = 29))
    np.savetxt(path + 'Eval_Sets/' + str(num_scen) + '_scenarios' + '/demand.txt', demand.reshape(5000*3,29))

In [28]:
p_sample_scenarios[350]

1.4240291640478235e-07

In [20]:
len(25*scenarios)

5000

In [16]:
path

'/home/dkabe/Model_brainstorming/SAA_Analysis/'

In [6]:
# 50 scenario confidence interval
batches = 30

In [7]:
# version 1
p_scen = np.loadtxt("/home/dkabe/Model_brainstorming/SAA_Analysis/p_scen.txt")
path = '/home/dkabe/Model_brainstorming/SAA_Analysis/'
scenarios = [50, 100, 150, 200, 250, 300, 350]

for scen in scenarios:
    gaps = []
    ub_averages = []
    ub_max = []
    objvals_lb = np.loadtxt(path + "Objectives/" + str(scen) + '_results.txt')
    avg_lb = (np.mean(objvals_lb))
    std_lb = (stdev(objvals_lb))
    lbwidth = std_lb/math.sqrt(batches)*2.045
    lbmin = round(avg_lb - lbwidth,2)
    print(avg_lb, lbmin)
    print('ci on lower bound = [', round(avg_lb-lbwidth,2), ',', round(avg_lb+lbwidth,2), ']')
    for batch in range(batches):
        objvals_ub = np.loadtxt(path + "Upper_Bounds/" + str(scen) + "_scenarios/" + str(scen) + "_" + str(batch) + "_UB_results.txt")
        avg_ub = (np.sum(p_scen*objvals_ub))
        std_ub = math.sqrt(np.sum(p_scen*((objvals_ub - avg_ub)**2)))
        ubwidth = std_ub/math.sqrt(1024)*1.96
        ubmax = round(avg_ub + ubwidth, 2)
        gap = round(100*((ubmax - lbmin)/lbmin),2)
        ub_averages.append(avg_ub)
        gaps.append(gap)
        ub_max.append(ubmax)
    min_gap = min(gaps)
    index = gaps.index(min_gap)
    print(ub_averages[index], ub_max[index], min_gap)
    

19311784.417 18226921.71
ci on lower bound = [ 18226921.71 , 20396647.12 ]
18914133.458780024 19293062.4 5.85
19727755.459666666 18856014.86
ci on lower bound = [ 18856014.86 , 20599496.06 ]
18914133.458780024 19293062.4 2.32
19125208.860666666 18151827.33
ci on lower bound = [ 18151827.33 , 20098590.39 ]
18914133.458780024 19293062.4 6.29
19203196.929 18574683.13
ci on lower bound = [ 18574683.13 , 19831710.73 ]
18914133.458780024 19293062.4 3.87
18669362.847333334 17970296.67
ci on lower bound = [ 17970296.67 , 19368429.02 ]
18914133.458780024 19293062.4 7.36
19322171.72633333 18960955.35
ci on lower bound = [ 18960955.35 , 19683388.1 ]
18914133.458780024 19293062.4 1.75
18922742.623666666 18411084.25
ci on lower bound = [ 18411084.25 , 19434401.0 ]
18914133.458780024 19293062.4 4.79


In [14]:
# version 2
path = '/home/dkabe/Model_brainstorming/SAA_Analysis/'
scenarios = [50, 100, 150, 200, 250, 300, 350]

for scen in scenarios:
    p_scen = np.loadtxt('/home/dkabe/Model_brainstorming/SAA_Analysis/Eval_Sets/' + str(scen) + '_scenarios/p_scen.txt') 
    gaps = []
    ub_averages = []
    ub_max = []
    objvals_lb = np.loadtxt(path + "Objectives/" + str(scen) + '_results.txt')
    avg_lb = (np.mean(objvals_lb))
    std_lb = (stdev(objvals_lb))
    lbwidth = std_lb/math.sqrt(batches)*2.045
    lbmin = round(avg_lb - lbwidth,2)
    #print(avg_lb, lbmin)
    #print('ci on lower bound = [', round(avg_lb-lbwidth,2), ',', round(avg_lb+lbwidth,2), ']')
    for batch in range(batches):
        objvals_ub = np.loadtxt(path + "Upper_Bounds_v2/" + str(scen) + "_scenarios/" + str(scen) + "_" + str(batch) + "_UB_results.txt")
        avg_ub = (np.sum(p_scen*objvals_ub))
        std_ub = math.sqrt(np.sum(p_scen*((objvals_ub - avg_ub)**2)))
        ubwidth = std_ub/math.sqrt(5000)*1.96
        ubmax = round(avg_ub + ubwidth, 2)
        gap = round(100*((ubmax - lbmin)/lbmin),2)
        ub_averages.append(avg_ub)
        gaps.append(gap)
        ub_max.append(ubmax)
    min_gap = min(gaps)
    index = gaps.index(min_gap)
    #print(ub_averages[index], ub_max[index], min_gap)
    print(avg_lb, "\t", ub_averages[index], "\t", lbmin, ub_max[index])

19311784.417 	 18605628.685058296 	 18226921.71 18712482.16
19727755.459666666 	 18951706.189399462 	 18856014.86 19084614.59
19125208.860666666 	 20626974.88149371 	 18151827.33 20871871.77
19203196.929 	 19082301.67707853 	 18574683.13 19239697.55
18669362.847333334 	 18002294.71588643 	 17970296.67 18276390.33
19322171.72633333 	 20659700.3648881 	 18960955.35 20854499.91
18922742.623666666 	 19421882.753571518 	 18411084.25 19623250.99


In [ ]:
# version 2 